In [16]:
# I am using CNN for text classification

import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [17]:
# load the training and test data
train  = pd.read_csv('SemEval2018-T3-train-taskA_emoji.txt', sep="\t")
test = pd.read_csv('SemEval2018-T3_gold_test_taskA_emoji.txt', sep="\t")

In [18]:
# combine training and test
combi = train.append(test, ignore_index=True)

In [19]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt
# remove twitter handles (@user)
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['Tweet text'], "@[\w]*")

In [20]:
combi

Tweet index  Label                                         Tweet text  \
0               1      1  Sweet United Nations video. Just in time for C...   
1               2      1  @mrdahl87 We are rumored to have talked to Erv...   
2               3      1  Hey there! Nice to see you Minnesota/ND Winter...   
3               4      0                3 episodes left I'm dying over here   
4               5      1  I can't breathe! was chosen as the most notabl...   
...           ...    ...                                                ...   
4596          780      0  If you drag yesterday into today, your tomorro...   
4597          781      0  Congrats to my fav @JennyBrew & her team & my ...   
4598          782      0  @allkpop: Jessica sheds tears at her fan signi...   
4599          783      1  #Irony: al jazeera is pro Anti - #GamerGate be...   
4600          784      0  #NOT ALL 👌 There good & bad in every occupatio...   

                                             tidy_tweet  
0     Sweet United Nations video. Just in time for C...  
1      We are rumored to have talked to Erv's agent....  
2     Hey there! Nice to see you Minnesota/ND Winter...  
3                   3 episodes left I'm dying over here  
4     I can't breathe! was chosen as the most notabl...  
...                                                 ...  
4596  If you drag yesterday into today, your tomorro...  
4597  Congrats to my fav  & her team & my birthplace...  
4598  : Jessica sheds tears at her fan signing event...  
4599  #Irony: al jazeera is pro Anti - #GamerGate be...  
4600  #NOT ALL 👌 There good & bad in every occupatio...  

[4601 rows x 4 columns]

In [21]:
# remove special characters, numbers, punctuations
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [22]:
# remove words less than 3 characters
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [23]:
# remove stopwords, stemming and lemmentization
def utils_preprocess_text(text, flg_stemm=True, flg_lemm =True, lst_stopwords=None ):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    #tokenization(convert from string to List)
    lst_text = text.split()
    #remove stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in
                   lst_stopwords]
        
     #stemming
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
        
    #Lemmentization
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
        
    # back to string from list
    text = " ".join(lst_text)
    return text

In [24]:
nltk.download('wordnet')
lst_stopwords = nltk.corpus.stopwords.words("english")
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: utils_preprocess_text(x, flg_stemm = False, flg_lemm=True))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cheng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
combi

Tweet index  Label                                         Tweet text  \
0               1      1  Sweet United Nations video. Just in time for C...   
1               2      1  @mrdahl87 We are rumored to have talked to Erv...   
2               3      1  Hey there! Nice to see you Minnesota/ND Winter...   
3               4      0                3 episodes left I'm dying over here   
4               5      1  I can't breathe! was chosen as the most notabl...   
...           ...    ...                                                ...   
4596          780      0  If you drag yesterday into today, your tomorro...   
4597          781      0  Congrats to my fav @JennyBrew & her team & my ...   
4598          782      0  @allkpop: Jessica sheds tears at her fan signi...   
4599          783      1  #Irony: al jazeera is pro Anti - #GamerGate be...   
4600          784      0  #NOT ALL 👌 There good & bad in every occupatio...   

                                             tidy_tweet  
0     sweet united nation video just time christmas ...  
1     rumored have talked agent angel asked about es...  
2                   there nice minnesota winter weather  
3                          episode left dying over here  
4     breathe chosen most notable quote year annual ...  
...                                                 ...  
4596  drag yesterday into today your tomorrow cannot...  
4597  congrats team birthplace team ohiostate claim ...  
4598  jessica shed tear signing event http http uegd...  
4599  irony jazeera anti gamergate because feminism ...  
4600  not there good every occupation race stop labe...  

[4601 rows x 4 columns]

In [26]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import glob

In [27]:
max_features = 100000 # Maximum Number of words we want to include in our dictionary
maxlen = 72 # No of words in question we want to create a sequence with
embed_size = 300# Size of word to vec embedding we are using

In [28]:
# split training into train and val
train_df, val_df = train_test_split(combi.iloc[:3816], test_size=0.1, random_state=2020)

In [29]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(combi['tidy_tweet'])
train_X = tokenizer.texts_to_sequences(train_df['tidy_tweet'])
val_X = tokenizer.texts_to_sequences(val_df['tidy_tweet'])
test_X = tokenizer.texts_to_sequences(combi.iloc[3817:]['tidy_tweet'])

# padding
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [30]:
train_y = train_df['Label'].values
val_y = val_df['Label'].values
test_y = test['Label'].values

print(train_df['Label'].value_counts())
print(val_df['Label'].value_counts())
print(test['Label'].value_counts())


0    1723
1    1711
Name: Label, dtype: int64
0    192
1    190
Name: Label, dtype: int64
0    473
1    311
Name: Label, dtype: int64


In [31]:
# we can load the pre-trained embedding model glove. I did not have enough PC resource for that and ignored it

# def get_coefs(word,*arr): 
#     return word, np.asarray(arr, dtype='float32')

# def get_embdedings_matrix(embeddings_index, word_index, nb_words = None):
#     all_embs = np.stack(embeddings_index.values())
#     print('Shape of Full Embeddding Matrix', all_embs.shape)
#     embed_dims = all_embs.shape[1]
#     emb_mean,emb_std = all_embs.mean(), all_embs.std()

#     #best to free up memory, given the size, which is usually ~3-4GB in memory
#     del all_embs
#     if nb_words is None:
#         nb_words = len(word_index)
#     else:
#         nb_words = min(nb_words, len(word_index))
    
#     embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_dims))
#     found_vectors = 0
#     words_not_found = []
#     for word, i in tqdm(word_index.items()):
#         if i >= nb_words: 
#             continue
#         embedding_vector = None
#         if word in embeddings_index:
#             embedding_vector = embeddings_index.get(word)
#         elif word.lower() in embeddings_index:
#             embedding_vector = embeddings_index.get(word.lower())
#         # for twitter check if the key is a hashtag
#         elif '#'+word.lower() in embeddings_index:
#             embedding_vector = embeddings_index.get('#'+word.lower())
            
#         if embedding_vector is not None: 
#             found_vectors += 1
#             embedding_matrix[i] = embedding_vector
#         else:
#             words_not_found.append((word, i))

#     print("% of Vectors found in Corpus", found_vectors / nb_words)
#     return embedding_matrix, words_not_found

# # Word 2 vec Embedding
# def load_glove(word_index):
# #     print('Loading Glove')
#     embed_file_path = 'glove840b300dtxt/glove.840B.300d.txt'
#     embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm(open(embed_file_path, 'r', encoding='utf-8')))
#     print("Built Embedding Index:", len(embeddings_index))
#     return get_embdedings_matrix(embeddings_index, word_index)

# glove_embed_matrix, words_not_found =  load_glove(tokenizer.word_index)

In [32]:
def model_cnn():
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, input_length=train_X.shape[1])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [33]:
tf.keras.backend.clear_session()
model = model_cnn()
# model = model_lstm_atten()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 72)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 72, 300)      30000000    input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 72, 300, 1)   0           embedding[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 72, 1, 36)    10836       reshape[0][0]                    
______________________________________________________________________________________________

In [34]:
def train_pred(model, epochs=2):
    filepath="weights_best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.0001, verbose=2)
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2, mode='auto')
    callbacks = [checkpoint, reduce_lr]
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y),callbacks=callbacks)
    model.load_weights(filepath)
    pred_val_y = model.predict([val_X], batch_size=256, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=256, verbose=0)
    return pred_val_y, pred_test_y

In [35]:
train_X = np.asarray(train_X)
train_y = np.asarray(train_y)
val_X = np.asarray(val_X)
val_y = np.asarray(val_y)
test_X = np.asarray(test_X)
test_y = np.asarray(test_y)
pred_val_y, pred_test_y = train_pred(model, epochs=8)

7/7 [==============================] - 12s 1s/step - loss: 0.6919 - accuracy: 0.5232 - val_loss: 0.6843 - val_accuracy: 0.5890

Epoch 00001: val_loss improved from inf to 0.68427, saving model to weights_best.h5
7/7 [==============================] - 10s 1s/step - loss: 0.6448 - accuracy: 0.7845 - val_loss: 0.6737 - val_accuracy: 0.6047

Epoch 00001: val_loss improved from 0.68427 to 0.67372, saving model to weights_best.h5
7/7 [==============================] - 10s 1s/step - loss: 0.5961 - accuracy: 0.8698 - val_loss: 0.6611 - val_accuracy: 0.6073

Epoch 00001: val_loss improved from 0.67372 to 0.66113, saving model to weights_best.h5
7/7 [==============================] - 11s 2s/step - loss: 0.5365 - accuracy: 0.8797 - val_loss: 0.6480 - val_accuracy: 0.6073

Epoch 00001: val_loss improved from 0.66113 to 0.64804, saving model to weights_best.h5
7/7 [==============================] - 9s 1s/step - loss: 0.4648 - accuracy: 0.8879 - val_loss: 0.6430 - val_accuracy: 0.6152

Epoch 00001: 

In [37]:
# To compute the best F1, we choose a suitable probability threshold
def f1_smart(y_true, y_pred):
    thresholds = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        res = metrics.f1_score(y_true, (y_pred > thresh).astype(int))
        thresholds.append([thresh, res])
        print("F1 score at threshold {0} is {1}".format(thresh, res))

    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh = thresholds[0][0]
    best_f1 = thresholds[0][1]
    print("Best threshold: ", best_thresh)
    return  best_f1, best_thresh

In [38]:
f1, threshold = f1_smart(val_y, pred_val_y)
print('Optimal F1: {} at threshold: {}'.format(f1, threshold))

F1 score at threshold 0.1 is 0.6643356643356644
F1 score at threshold 0.11 is 0.6654991243432574
F1 score at threshold 0.12 is 0.6654991243432574
F1 score at threshold 0.13 is 0.6654991243432574
F1 score at threshold 0.14 is 0.6654991243432574
F1 score at threshold 0.15 is 0.6654991243432574
F1 score at threshold 0.16 is 0.6654991243432574
F1 score at threshold 0.17 is 0.6678383128295254
F1 score at threshold 0.18 is 0.6690140845070423
F1 score at threshold 0.19 is 0.6690140845070423
F1 score at threshold 0.2 is 0.6690140845070423
F1 score at threshold 0.21 is 0.6690265486725664
F1 score at threshold 0.22 is 0.6702127659574467
F1 score at threshold 0.23 is 0.6714031971580818
F1 score at threshold 0.24 is 0.6737967914438504
F1 score at threshold 0.25 is 0.6810810810810811
F1 score at threshold 0.26 is 0.6823956442831216
F1 score at threshold 0.27 is 0.676416819012797
F1 score at threshold 0.28 is 0.6789667896678966
F1 score at threshold 0.29 is 0.6754221388367729
F1 score at threshold 0

In [39]:
pred_test_y = (pred_test_y >threshold).astype(int)
res = metrics.f1_score(test_y, pred_test_y)
print('the F1 score of tweet classification:{}'.format(res))

the F1 score of tweet classification:0.6414662084765177
